In [ ]:
import os
import sys
import platform
from datetime import datetime
from dotenv import load_dotenv

# Import the new LangGraph Brain
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

# Import our Hybrid Switcher logic
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from langchain_core.tools import tool

load_dotenv()

# --- 1. SETUP THE BRAIN ---
def get_llm():
    llm_type = os.getenv("LLM_TYPE", "ollama")
    if llm_type == "groq":
        print(f"🚀 Brain: Groq (Fast)")
        return ChatGroq(model="llama-3.1-8b-instant", temperature=0)
    else:
        print(f"🏠 Brain: Local Ollama")
        return ChatOllama(model="llama3.1", temperature=0)

# --- 2. DEFINE THE TOOLS (THE HANDS) ---
@tool
def get_current_time():
    """Returns the current time in H:M:S format."""
    now = datetime.now()
    return now.strftime("%H:%M:%S")

@tool
def multiply_numbers(a: float, b: float):
    """Multiplies two numbers. Use this for ANY math calculation."""
    return a * b

@tool
def get_system_info():
    """Returns the type of OS"""
    system_info = platform.system()
    return system_info

# --- 3. BUILD THE AGENT ---
llm = get_llm()
tools = [get_current_time, multiply_numbers, get_system_info]

# This Function creates the "Loop" automatically
# It connects the LLM -> Tool -> Output -> LLM
agent = create_react_agent(llm, tools)

# --- 4. RUN THE AGENT ---
def run_agent(query):
    print(f"\nUser: {query}")
    print("🤖 Agent is thinking...")
    
    # We pass the input as a "message"
    result = agent.invoke({"messages": [HumanMessage(content=query)]})
    
    # The result contains the whole history (User, AI, Tool Output, AI Final)
    # We just want the last message (the final answer)
    last_message = result["messages"][-1]
    print(f"AI: {last_message.content}")

# Test 1: Simple Math
run_agent("What is 25 times 12?")

# Test 2: Context + Tools
run_agent("What time is it right now? Also, take the current hour and multiply it by 10.")


run_agent("What operating system am I running and what is 50 * 5?")

🚀 Brain: Groq (Fast)


C:\Users\OHB\AppData\Local\Temp\ipykernel_7008\786208734.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)



User: What is 25 times 12?
🤖 Agent is thinking...
AI: The current time is 17:49:45, 25 times 12 is 300.0, and the system is running on Windows.

User: What time is it right now? Also, take the current hour and multiply it by 10.
🤖 Agent is thinking...
AI: The current time is 17:49:45, the current hour multiplied by 10 is 170, and the type of OS is Windows.

User: What operating system am I running and what is 50 * 5?
🤖 Agent is thinking...
AI: The operating system is Windows and 50 * 5 is 250.
